<a href="https://colab.research.google.com/github/Andrian0s/ML4NLP1-2024-Tutorial-Notebooks/blob/main/exercises/ex5/ex5_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 5: LLM Prompting and Prompt Engineering Part 2

In part 2, we experiment with prompting instruction-tuned Large Language Models (LLMs), and evaluate their performance on a linguistic annotation task involving structured outputs.

The goal of this assignment is to gain some experience working with instruction-tuned LLMs. To this end, you will learn how to

- query an instruction-tuned LLM with default chat templates (`Llama-3.2-3B-Instruct`)
- parse LLM outputs for structured responses using JSON and `Pydantic`
- implement error handling for edge cases where the model fails to output the expected data format.

The task we use for this purpose is a simple Tokenization and Part-of-Speech tagging task using data taken from Universal Dependencies.

To facilitate working with LLMs, we will again use the Unsloth library. Note that Unsloth provides both freeware and closed-source proprietary software. For our purposes, the freeware is sufficient! For more information on Unsloth, see the docs here.

This notebook is adapted from [this example](https://colab.research.google.com/drive/1T5-zKWM_5OD21QHwXHiV9ixTRR7k3iB9?usp=sharing) by Unsloth.


### NOTE: Expected execution times
We have provided expected execution times throughout the notebook as a guide. These are intended to be approximate, but should give you some idea for what to expect. If your runtimes far exceed these expected execution times, you may want to consider modifying your approach. These are denoted with ⌛ .

### NOTE: GPU Usage
It is expected that you load the model onto a GPU for inference. For other parts of the code, such as data preparation, a GPU is not necessary. To avoid waiting for resources unnecessarily, we recommend doing as much as you can on a CPU instance and change the runtime type as necessary. We've highlight the cells that need a GPU with ⚡

## 1) Installing dependencies

In [ ]:
%%capture
!pip install levenshtein
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# check unsloth version
expected_version = '2024.10.2'
unsloth_version = !pip list | grep -P 'unsloth\s+' | grep -Po '\S+$'
if unsloth_version[0] != expected_version:
    print(f"Warning! Found Unsloth version {unsloth_version[0]} but expected {expected_version}.")

# check python version
import sys
print(sys.version)

# check gpu info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# check RAM info
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Warning! Found Unsloth version 2024.11.10 but expected 2024.10.2.
3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
Tue Nov 26 20:13:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                        

## 2) Model Loading

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# Note, here we specify the instruction-tuned version of Llama-3.2-3B
model_name = "unsloth/Llama-3.2-3B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )


FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Llam

## 3) Data Loading and Preparation

In [ ]:
# load data
import random
import pandas as pd

seed = 42

random.seed(seed)

dataset_url = "https://raw.githubusercontent.com/tannonk/prompting_exercise/refs/heads/main/data/en_ewt-ud-dev-pos.json"
df = pd.read_json(dataset_url, lines=True)

# For each input sentence, we'll build the target as a list of dictionaries containing keys for the token and pos tag. This is what we want our LLM annotator to predict.
df['target'] = df.apply(lambda x: [{'token': token, 'pos': pos} for token, pos in zip(x['tokens'].split(), x['upos'].split())], axis=1)

# We'll sample 100 items for testing purposes
test_data = df.sample(n=100, random_state=seed)
train_data = df.drop(test_data.index)

print(f"Train data: {len(train_data)}")
print(f"Test data: {len(test_data)}")

test_data.head()


Train data: 1408
Test data: 100


,sentence,tokens,upos,xpos,target
578,"""...there is no companion quite so devoted, so...",""" ... there is no companion quite so devoted ,...",PUNCT PUNCT PRON VERB DET NOUN ADV ADV ADJ PUN...,"`` , EX VBZ DT NN RB RB JJ , RB JJ , RB JJ CC ...","[{'token': '""', 'pos': 'PUNCT'}, {'token': '....."
1146,"Great computer repair store, highly recommended.","Great computer repair store , highly recommend...",ADJ NOUN NOUN NOUN PUNCT ADV VERB PUNCT,"JJ NN NN NN , RB VBN .","[{'token': 'Great', 'pos': 'ADJ'}, {'token': '..."
382,You wear your heart on your sleeve ... and sin...,You wear your heart on your sleeve ... and sin...,PRON VERB PRON NOUN ADP PRON NOUN PUNCT CCONJ ...,"PRP VBP PRP$ NN IN PRP$ NN , CC IN PRP VBP DT ...","[{'token': 'You', 'pos': 'PRON'}, {'token': 'w..."
583,for Books that Speak for Themselves....,for Books that Speak for Themselves ....,ADP NOUN PRON VERB ADP PRON PUNCT,"IN NNS WDT VBP IN PRP ,","[{'token': 'for', 'pos': 'ADP'}, {'token': 'Bo..."
966,yuck !!,yuck !!,INTJ PUNCT,UH .,"[{'token': 'yuck', 'pos': 'INTJ'}, {'token': '..."


In [ ]:
# Extracting UPOS tags from the `upos` column by splitting the strings into individual tags
upos_tags = test_data['upos'].str.split(expand=True).stack()

# Calculating the frequency of each UPOS tag
upos_tag_counts = upos_tags.value_counts()

# Displaying the result
print("Frequency of each UPOS tag:")
print(upos_tag_counts)


Frequency of each UPOS tag:
NOUN     219
PUNCT    146
VERB     124
ADJ      111
PRON     106
ADP       99
DET       88
AUX       72
ADV       69
PROPN     69
CCONJ     41
PART      17
NUM       16
SCONJ     13
INTJ       6
SYM        3
X          1
Name: count, dtype: int64


### TODO: Inspect and describe the data

📝❓ What are the fields and their corresponding values in the dataframe?

📝❓ What is the difference between `upos` and `xpos`?

📝❓ What is the distribution of `upos` labels in the `test_data`?

### TODO: Define the basic `PromptTemplate`

Note, you can reuse the solution from part 1 of this exercise here.

In [ ]:
class PromptTemplate:
    def __init__(self, task_description, bos=tokenizer.bos_token, eos="\n\n"):
        self.task_description = task_description
        self.bos = bos # Used for training prompt formatting
        self.eos = eos # Used for training prompt formatting

    def chat_template_prompt(self,prompt_data,examples):
      chat_list = []
      for _,row in prompt_data.iterrows():
        few_shot_examples = examples.sample(n=4, random_state=42).to_dict(orient="records")
        chat_prompt = [{"role": "assistant", "content": self.task_description}]
        for example in few_shot_examples:
          chat_prompt.append({"role": "user", "content": example['input']})
          chat_prompt.append({"role": "assistant", "content": json.dumps(example['output'])})
        chat_prompt.append({"role": "user", "content": row['input']})
        print(chat_prompt)
        templated_chat = tokenizer.apply_chat_template(chat_prompt, tokenize=False)
        print(templated_chat)
        chat_list.append(templated_chat)
      return chat_list

## 3.2 ChatTemplates

Instruction-tuned models are typically finetuned using a predefined `ChatTemplate`.
This means that when using them for inference, it is important that we use the correct `ChatTemplate` in order to avoid "confusing" the model.

You can find more information about model `ChatTemplates` for Huggingface models [here](https://huggingface.co/docs/transformers/en/chat_templating).


In [ ]:
from unsloth.chat_templates import get_chat_template

# load the chat_template from unsloth (note, the logic is similar when using native Huggingface, but here we're using Unsloth!)
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1", # for Llama-3.1 and Llama-3.2 models
)

# Inspect the template (note, it looks more complicated than it is!)
print(tokenizer.chat_template)


{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 July 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content'] %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>

" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython
" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "

"}}
{%- endif %}
{{- "

In [ ]:
examples = train_data
columns_to_keep = {'sentence':'input','target':'output'}
examples = train_data[list(columns_to_keep.keys())].rename(columns=columns_to_keep)

prompt_data = test_data[list(columns_to_keep.keys())].rename(columns=columns_to_keep)
prompt_data

,input,output
578,"""...there is no companion quite so devoted, so...","[{'token': '""', 'pos': 'PUNCT'}, {'token': '....."
1146,"Great computer repair store, highly recommended.","[{'token': 'Great', 'pos': 'ADJ'}, {'token': '..."
382,You wear your heart on your sleeve ... and sin...,"[{'token': 'You', 'pos': 'PRON'}, {'token': 'w..."
583,for Books that Speak for Themselves....,"[{'token': 'for', 'pos': 'ADP'}, {'token': 'Bo..."
966,yuck !!,"[{'token': 'yuck', 'pos': 'INTJ'}, {'token': '..."
...,...,...
859,"certainly not ""normal"" photography... forensic...","[{'token': 'certainly', 'pos': 'ADV'}, {'token..."
486,It would be appreciated if you could advice me...,"[{'token': 'It', 'pos': 'PRON'}, {'token': 'wo..."
1214,Store is on the small side and atmosphere is j...,"[{'token': 'Store', 'pos': 'NOUN'}, {'token': ..."
1375,They do have a good selection of fabric and no...,"[{'token': 'They', 'pos': 'PRON'}, {'token': '..."


### TODO: Prepare your inputs using the `ChatTemplate` for the model.

Note, you should be able to drop your custom `PromptTemplate` string into the model's default `ChatTemplate`.


In [ ]:
import json

task_description = "Do tokenization and part of Speech tagging with following sentences."

prompt_template = PromptTemplate(task_description)

chat_list = prompt_template.chat_template_prompt(prompt_data,examples)

[{'role': 'assistant', 'content': 'Do tokenization and part of Speech tagging with following sentences.'}, {'role': 'user', 'content': 'Not sure if I am going to buy 17" or 16" wheels for the winter.'}, {'role': 'assistant', 'content': '[{"token": "Not", "pos": "PART"}, {"token": "sure", "pos": "ADJ"}, {"token": "if", "pos": "SCONJ"}, {"token": "I", "pos": "PRON"}, {"token": "am", "pos": "AUX"}, {"token": "going", "pos": "VERB"}, {"token": "to", "pos": "PART"}, {"token": "buy", "pos": "VERB"}, {"token": "17", "pos": "NUM"}, {"token": "\\"", "pos": "NOUN"}, {"token": "or", "pos": "CCONJ"}, {"token": "16", "pos": "NUM"}, {"token": "\\"", "pos": "NOUN"}, {"token": "wheels", "pos": "NOUN"}, {"token": "for", "pos": "ADP"}, {"token": "the", "pos": "DET"}, {"token": "winter", "pos": "NOUN"}, {"token": ".", "pos": "PUNCT"}]'}, {'role': 'user', 'content': 'Privacy in kerala,help pls..?'}, {'role': 'assistant', 'content': '[{"token": "Privacy", "pos": "NOUN"}, {"token": "in", "pos": "ADP"}, {"to

## 4) Inference Pipeline

### TODO: Define a function to run inference efficiently with an LLM

Note, you can use the same inference function from part 1 of this exercise here!

In [ ]:
# Set up our inference pipeline for generation

# We'll set some default generation args that we'll pass to our inference function
# Following best practices, we'll use Pydantic class which helps with validation.
from pydantic import BaseModel

class Generation_Args(BaseModel):
    max_new_tokens: int
    temperature: float
    top_k: int
    top_p: float
    repetition_penalty: float
    do_sample: bool
    min_p: float
    num_return_sequences: int

# Here are some default generation args
generation_args = Generation_Args(
    max_new_tokens = 1024, # note, for this task, we're setting the max_new_tokens to be more appropriate
    temperature = 1.0,
    top_k = 0,
    top_p = 1.0,
    repetition_penalty = 1.0,
    do_sample = True,
    use_cache = True,
    min_p = 0.1,
    num_return_sequences = 1
)


def run_batched_inference(prompts, model, tokenizer, batch_size=10, generation_args=generation_args):
    """
    Runs batched inference on a list of prompts using a given model and tokenizer.

    Set the batch_size to control the number of prompts processed in each batch.
    Depending on the length of your prompts and model size the batch size may need to be adjusted.

    Args:
        prompts (list[str]): List of prompts that are passed to the model
        model (): The model used for generation
        tokenizer (): The tokenizer used for encoding and decoding the prompts
        batch_size (int): Number of prompts to run batched inference on.

    Returns:
        List[str] containing generated outputs.
    """

    outputs = []

    # Process prompts in batches
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i + batch_size]

        # Tokenize inputs
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True)

        # Generate outputs using the model
        raw_outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=generation_args.max_new_tokens,
            temperature=generation_args.temperature,
            top_k=generation_args.top_k,
            top_p=generation_args.top_p,
            repetition_penalty=generation_args.repetition_penalty,
            do_sample=generation_args.do_sample,
            num_return_sequences=generation_args.num_return_sequences
        )

        # Decode generated tokens to text
        batch_outputs = tokenizer.batch_decode(raw_outputs, skip_special_tokens=True)
        outputs.extend(batch_outputs)


    # TODO: implement the logic for efficient inference with LLM

    return outputs

## 4.1) Run Inference

### TODO: run inference!

⌛ 10-20 mins

⚡ GPU

In [ ]:
output = run_batched_inference(chat_list,model,tokenizer,1)

In [ ]:
output

['system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nassistant\n\nDo tokenization and part of Speech tagging with following sentences.user\n\nNot sure if I am going to buy 17" or 16" wheels for the winter.assistant\n\n[{"token": "Not", "pos": "PART"}, {"token": "sure", "pos": "ADJ"}, {"token": "if", "pos": "SCONJ"}, {"token": "I", "pos": "PRON"}, {"token": "am", "pos": "AUX"}, {"token": "going", "pos": "VERB"}, {"token": "to", "pos": "PART"}, {"token": "buy", "pos": "VERB"}, {"token": "17", "pos": "NUM"}, {"token": "\\"", "pos": "NOUN"}, {"token": "or", "pos": "CCONJ"}, {"token": "16", "pos": "NUM"}, {"token": "\\"", "pos": "NOUN"}, {"token": "wheels", "pos": "NOUN"}, {"token": "for", "pos": "ADP"}, {"token": "the", "pos": "DET"}, {"token": "winter", "pos": "NOUN"}, {"token": ".", "pos": "PUNCT"}]user\n\nPrivacy in kerala,help pls..?assistant\n\n[{"token": "Privacy", "pos": "NOUN"}, {"token": "in", "pos": "ADP"}, {"token": "kerala", "pos": "PROPN"}, {"token": 

## 5) Structured output validation

LLMs output text. But in practice, we often want structured data that we can process further with other automatic processes.

For this purpose, JSON is a good target data structure.


### TODO: Define a processing pipeline that extracts and validates the JSON response from the LLM.

Hint: For this you should use a combination of [`Regex`](https://www.w3schools.com/python/python_regex.asp) and [`Pydantic`](https://docs.pydantic.dev/latest/).

The output should be a valid json object with the following structure:

```json
[
    {"token": "there", "pos": "DET"}, # each dict contains a token and its corresponding POS-Tag.
    {"token": "is", "pos": "VERB"},
    {"token": "no", "pos": "ADJ"},
    {"token": "companion", "pos": "NOUN"},
    {"token": "quite", "pos": "ADV"},
    {"token": "so", "pos": "ADV"},
    {"token": "devoted", "pos": "ADV"},
    {"token": "so", "pos": "ADV"},
    ...
]
```

In [ ]:
import re
import json
from typing import List, Dict

def process_llm_output(outputs: List[str]) -> List[Dict]:
    """
    Processes the LLM output to extract and validate JSON responses using regular expressions.

    Args:
        outputs (List[str]): List of raw LLM outputs containing JSON-like responses.

    Returns:
        List[Dict]: A valid JSON object containing all the token-pos pairs.
    """
    result = []

    # Regular expression to extract "token": "..." and "pos": "..."
    pair_pattern = re.compile(r'"token":\s*"([^{}].*?)",\s*"pos":\s*"([^{}].*?)"')

    for idx, output in enumerate(outputs):
        # Extract the last line that should contain JSON-like content
        json_candidate = output.split('\n')[-1].strip()

        # Use regex to find all matching token-pos pairs
        matches = pair_pattern.findall(json_candidate)

        if matches:
            # Convert matches into a list of dictionaries
            parsed_json = [{"token": token, "pos": pos} for token, pos in matches]
            result.append(parsed_json)
        else:
            print(f"Warning: No valid token-pos pairs found in output index {idx}. Appending an empty list.")
            result.append([])  # Append empty list for invalid cases

    return result

processed_outputs = process_llm_output(output)

In [ ]:
processed_outputs

[[{'token': '", "pos": "NOUN"}, {"token": "mesmerizing', 'pos': 'ADJ'},
  {'token': 'as', 'pos': 'CONJ'},
  {'token': 'a', 'pos': 'DET'},
  {'token': 'rat', 'pos': 'NOUN'}],
 [{'token': 'Great', 'pos': 'ADV'},
  {'token': 'computer', 'pos': 'NOUN'},
  {'token': 'repair', 'pos': 'NOUN'},
  {'token': 'store', 'pos': 'NOUN'},
  {'token': ', ', 'pos': 'PUNCT'},
  {'token': 'highly', 'pos': 'ADVB'},
  {'token': 'recommended', 'pos': 'ADVB'}],
 [{'token': 'take', 'pos': 'AUX'},
  {'token': 'care', 'pos': 'NOUN'},
  {'token': '.', 'pos': 'PUNCT'}],
 [{'token': 'for', 'pos': 'PREP'},
  {'token': 'Books', 'pos': 'NOUN'},
  {'token': 'that', 'pos': 'DET'},
  {'token': 'Speak', 'pos': 'NOUN'},
  {'token': 'for', 'pos': 'preposition'},
  {'token': 'for', 'pos': 'preposition'},
  {'token': ' THEMSELVES', 'pos': ' Volvo '},
  {'token': '.', 'pos': 'PUNCT'}],
 [],
 [{'token': 'Great', 'pos': 'ADJ'},
  {'token': 'School', 'pos': 'NOUN'},
  {'token': '!', 'pos': 'PUNCT'}],
 [],
 [{'token': 'Definitely'

## 6) *Evaluation*

In [ ]:
# Below is some boilerplate evaluation code. You should not need to make any changes here.

import Levenshtein
import numpy as np
from typing import List, Dict

def evaluate_instance(target: List[Dict], prediction: List[Dict]):
    """
    Evaluates the accuracy of tokenization and part-of-speech (POS) tagging between a target and a predicted sequence.

    Args:
        target (List[Dict]): A list of dictionaries representing the target tokens and POS tags.
        prediction (List[Dict]): A list of dictionaries representing the predicted tokens and POS tags.

    Returns:
        dict: A dictionary containing the token-level accuracy ('Token Acc') and POS accuracy ('POS Acc').
    """

    # If there is no prediction, return zero accuracies
    if prediction is None:
        return {'Token Acc': 0, 'POS Acc': 0}

    # Extract tokens and POS tags from the target and prediction lists
    target_tokens = [item['token'] for item in target]
    target_pos = [item['pos'] for item in target]
    pred_tokens = [item['token'] for item in prediction]
    pred_pos = [item['pos'] for item in prediction]

    # Get alignment operations between the target and predicted tokens using Levenshtein.opcodes()
    opcodes = Levenshtein.opcodes(target_tokens, pred_tokens)

    # Initialize aligned lists to store tokens and POS tags after alignment
    aligned_target_tokens = []
    aligned_target_pos = []
    aligned_pred_tokens = []
    aligned_pred_pos = []

    # Iterate over each operation in the alignment
    for tag, i1, i2, j1, j2 in opcodes:
        # "equal" means the tokens in this range are identical in both sequences
        if tag == 'equal':
            aligned_target_tokens.extend(target_tokens[i1:i2])
            aligned_target_pos.extend(target_pos[i1:i2])
            aligned_pred_tokens.extend(pred_tokens[j1:j2])
            aligned_pred_pos.extend(pred_pos[j1:j2])
        # "replace" means tokens in this range are different between the target and prediction
        elif tag == 'replace':
            aligned_target_tokens.extend(target_tokens[i1:i2])
            aligned_target_pos.extend(target_pos[i1:i2])
            aligned_pred_tokens.extend(pred_tokens[j1:j2])
            aligned_pred_pos.extend(pred_pos[j1:j2])
        # "insert" means tokens were added in the prediction that are not in the target
        elif tag == 'insert':
            aligned_target_tokens.extend(['<MISSING>'] * (j2 - j1))  # Add placeholders for missing target tokens
            aligned_target_pos.extend(['<MISSING>'] * (j2 - j1))      # Add placeholders for missing target POS tags
            aligned_pred_tokens.extend(pred_tokens[j1:j2])
            aligned_pred_pos.extend(pred_pos[j1:j2])
        # "delete" means tokens are present in the target but missing in the prediction
        elif tag == 'delete':
            aligned_target_tokens.extend(target_tokens[i1:i2])
            aligned_target_pos.extend(target_pos[i1:i2])
            aligned_pred_tokens.extend(['<MISSING>'] * (i2 - i1))    # Add placeholders for missing predicted tokens
            aligned_pred_pos.extend(['<MISSING>'] * (i2 - i1))       # Add placeholders for missing predicted POS tags

    # Calculate token-level accuracy
    # We only consider positions where both target and prediction have valid tokens (i.e., not '<MISSING>')
    correct_tokens = [
        1 if tgt == pred else 0
        for tgt, pred in zip(aligned_target_tokens, aligned_pred_tokens)
        if tgt != '<MISSING>' and pred != '<MISSING>'
    ]
    token_accuracy = np.mean(correct_tokens) if correct_tokens else 0

    # Calculate POS accuracy
    # Only consider positions where tokens match and are not '<MISSING>'
    correct_pos = [
        1 if tgt_pos == pred_pos else 0
        for tgt_tok, pred_tok, tgt_pos, pred_pos in zip(aligned_target_tokens, aligned_pred_tokens, aligned_target_pos, aligned_pred_pos)
        if tgt_tok == pred_tok and tgt_tok != '<MISSING>'
    ]
    pos_accuracy = np.mean(correct_pos) if correct_pos else 0

    return {'Token Acc': token_accuracy, 'POS Acc': pos_accuracy}

def get_results(test_data: pd.DataFrame, processed_outputs: List[List[Dict]]):
    """
    Returns a summary dataframe by taking the average of the all results for tokenization and pos-tagging.
    """
    results = []
    for i in range(len(processed_outputs)):
        results.append(evaluate_instance(test_data.iloc[i]['target'], processed_outputs[i]))

    results = pd.DataFrame(results).mean()
    return results

In [ ]:
# To get the results, you should be able to pass your test_data DataFrame and the processed_outputs from above...
get_results(test_data, processed_outputs)

,0
Token Acc,0.840994
POS Acc,0.644840


## 7) Manipulating the system prompt

The system prompt is part of the `ChatTemplate` that can help to steer the model.


### TODO: Customise the system prompt for the intended task and re-run inference

Note, this is an experiment. You should try a few different system prompts and report the resulting performance in your report.


📝❓ What was the best system prompt you considered?

📝❓ Were you able to improve the performance by manipulating the system prompt? Please discuss.

⌛ 10-20 mins (per experiment run)

⚡ GPU

In [ ]:
# Focus on Accurate Annotation
task_description_1 = 'You are an advanced language model designed to analyze text with high precision. Your task is to perform tokenization and part-of-speech (POS) tagging on the provided input sentences. For each word or token in the sentence, annotate it with the correct POS tag based on linguistic conventions.'

# Focus on JSON Structured Output
task_description_2 = 'Do tokenization and POS on following sentences, the output should be in a json format'

prompt_template_1 = PromptTemplate(task_description_1)

prompt_template_2 = PromptTemplate(task_description_2)

chat_list_1 = prompt_template_1.chat_template_prompt(prompt_data,examples)

chat_list_2 = prompt_template_2.chat_template_prompt(prompt_data,examples)

Streaming output truncated to the last 5000 lines.
just call me on my cell phone.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

[{"token": "just", "pos": "ADV"}, {"token": "call", "pos": "VERB"}, {"token": "me", "pos": "PRON"}, {"token": "on", "pos": "ADP"}, {"token": "my", "pos": "PRON"}, {"token": "cell", "pos": "NOUN"}, {"token": "phone", "pos": "NOUN"}, {"token": ".", "pos": "PUNCT"}]<|eot_id|><|start_header_id|>user<|end_header_id|>

Any information about CRAZY HORSE SCULPTURE?<|eot_id|>
[{'role': 'assistant', 'content': 'You are an advanced language model designed to analyze text with high precision. Your task is to perform tokenization and part-of-speech (POS) tagging on the provided input sentences. For each word or token in the sentence, annotate it with the correct POS tag based on linguistic conventions.'}, {'role': 'user', 'content': 'Not sure if I am going to buy 17" or 16" wheels for the winter.'}, {'role': 'assistant', 'content': '[{"token": "Not", "pos": "PART

In [ ]:
output_1 = run_batched_inference(chat_list_1,model,tokenizer,1)

In [ ]:
processed_outputs_1 = process_llm_output(output_1)

In [ ]:
get_results(test_data, processed_outputs_1)

,0
Token Acc,0.878076
POS Acc,0.656337


In [ ]:
output_2 = run_batched_inference(chat_list_2,model,tokenizer,1)

In [ ]:
processed_outputs_2 = process_llm_output(output_2)

In [ ]:
get_results(test_data, processed_outputs_2)

,0
Token Acc,0.856521
POS Acc,0.615859


---

## 8) Lab report

📝❓ Write your lab report here addressing all questions in the notebook

# Report on POS Tagging and Tokenization

## Answer 1: Fields and Their Corresponding Values in the DataFrame

| **Field**   | **Description**                                                                                      |
|-------------|------------------------------------------------------------------------------------------------------|
| `sentence`  | The original input text.                                                                             |
| `tokens`    | The sentence in a more formatted way, broken down into individual tokens.                            |
| `upos`      | Universal POS tagging corresponding to the tokens in the sentence.                                   |
| `xpos`      | Language-specific POS tagging corresponding to the tokens in the sentence.                           |
| `target`    | A dictionary mapping each token to its corresponding UPOS tag.                                        |

---

## Answer 2: Difference Between UPOS and XPOS

| **Tag Type** | **Description**                                                                                   |
|--------------|---------------------------------------------------------------------------------------------------|
| `UPOS`       | Universal POS tagging. It is language-independent and applies consistently across all languages.  |
| `XPOS`       | Language-specific POS tagging. It varies based on the grammar and conventions of a specific language. |

---

## Answer 3: Distribution of UPOS Labels in `test_data`

| **UPOS Tag** | **Frequency** |
|--------------|---------------|
| `NOUN`       | 219           |
| `PUNCT`      | 146           |
| `VERB`       | 124           |
| `ADJ`        | 111           |
| `PRON`       | 106           |
| `ADP`        | 99            |
| `DET`        | 88            |
| `AUX`        | 72            |
| `ADV`        | 69            |
| `PROPN`      | 69            |
| `CCONJ`      | 41            |
| `PART`       | 17            |
| `NUM`        | 16            |
| `SCONJ`      | 13            |
| `INTJ`       | 6             |
| `SYM`        | 3             |
| `X`          | 1             |

---

## Answer 4: Best System Prompt Considered

The best system prompt was **`task_description_1`**, as it achieved the highest scores for both token accuracy and POS accuracy.

---

## Answer 5: Performance Improvement by Manipulating the System Prompt

Yes, manipulating the system prompt slightly improved the performance. For **`task_description_1`**, the prompt explicitly instructs the model to focus on high accuracy, resulting in the best performance among the prompts tested.

| **System Prompt**        | **Token Accuracy** | **POS Accuracy** |
|--------------------------|--------------------|------------------|
| `task_description`       | 0.840994           | 0.644840         |
| `task_description_1`     | 0.878076           | 0.656337         |
| `task_description_2`     | 0.856521           | 0.615859         |

### Task Descriptions:

- **`task_description`:** `"Do tokenization and part of Speech tagging with following sentences."`

- **`task_description_1`:**
  `"You are an advanced language model designed to analyze text with high precision. Your task is to perform tokenization and part-of-speech (POS) tagging on the provided input sentences. For each word or token in the sentence, annotate it with the correct POS tag based on linguistic conventions."`

- **`task_description_2`:**
  `"Do tokenization and POS on following sentences, the output should be in a json format."`

---

### Summary

By providing a clear and precise system prompt (e.g., **`task_description_1`**), the model's performance improved both in token and POS accuracy, highlighting the importance of careful prompt engineering.

